In [16]:
import pandas as pd
import numpy as np

import os
import re
import nltk
import string
import sys
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer


In [17]:
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\panka\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\panka\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
df=pd.read_csv('data/ecom_product_data.csv')

In [7]:
df.shape

(147702, 14)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 147702 entries, 0 to 147701
Data columns (total 14 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   item_id        147702 non-null  object
 1   marketplace    147702 non-null  object
 2   country        147702 non-null  object
 3   main_image_id  147127 non-null  object
 4   domain_name    147702 non-null  object
 5   bullet_point   113481 non-null  object
 6   item_keywords  107566 non-null  object
 7   material       46939 non-null   object
 8   brand          118588 non-null  object
 9   color          98503 non-null   object
 10  item_name      122734 non-null  object
 11  model_name     74671 non-null   object
 12  model_number   124056 non-null  object
 13  product_type   147702 non-null  object
dtypes: object(14)
memory usage: 15.8+ MB


In [9]:
df.isna().sum()

item_id               0
marketplace           0
country               0
main_image_id       575
domain_name           0
bullet_point      34221
item_keywords     40136
material         100763
brand             29114
color             49199
item_name         24968
model_name        73031
model_number      23646
product_type          0
dtype: int64

In [10]:
text_cols = [
    "bullet_point", "item_keywords", "material",
    "brand", "color", "item_name",
    "model_name", "model_number"
]

for col in text_cols:
    df[col] = df[col].fillna("unknown")

df["main_image_id"] = df["main_image_id"].fillna("no_image")


In [11]:
df.isna().sum()

item_id          0
marketplace      0
country          0
main_image_id    0
domain_name      0
bullet_point     0
item_keywords    0
material         0
brand            0
color            0
item_name        0
model_name       0
model_number     0
product_type     0
dtype: int64

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 147702 entries, 0 to 147701
Data columns (total 14 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   item_id        147702 non-null  object
 1   marketplace    147702 non-null  object
 2   country        147702 non-null  object
 3   main_image_id  147702 non-null  object
 4   domain_name    147702 non-null  object
 5   bullet_point   147702 non-null  object
 6   item_keywords  147702 non-null  object
 7   material       147702 non-null  object
 8   brand          147702 non-null  object
 9   color          147702 non-null  object
 10  item_name      147702 non-null  object
 11  model_name     147702 non-null  object
 12  model_number   147702 non-null  object
 13  product_type   147702 non-null  object
dtypes: object(14)
memory usage: 15.8+ MB


In [19]:
df.head()

,item_id,marketplace,country,main_image_id,domain_name,bullet_point,item_keywords,material,brand,color,item_name,model_name,model_number,product_type
0,B07T6RZ2CM,Amazon,IN,71dZhpsferL,amazon.in,3D Printed Hard Back Case Mobile Cover for Len...,mobile cover back cover mobile case phone case...,unknown,Amazon Brand - Solimo,Others,Amazon Brand - Solimo Designer Couples Sitting...,Lenovo K4 Note,gz8115-SL40423,CELLULAR_PHONE_CASE
1,B07T2JY31Y,Amazon,IN,71vX7qIEAIL,amazon.in,3D Printed Hard Back Case Mobile Cover for Son...,mobile cover back cover mobile case phone case...,Wood,Amazon Brand - Solimo,others,Amazon Brand - Solimo Designer Leaf on Wood 3D...,Sony Xperia Z1 L39H,gz8056-SL40528,CELLULAR_PHONE_CASE
2,B0849YGSCZ,Amazon,AE,A1EZF-2mB5L,amazon.ae,unknown,small de fur rooms navidad woven girls shag pa...,unknown,Stone & Beam,unknown,Stone & Beam Contemporary Doily Wool Farmhouse...,unknown,I59I8044IVYGRYC00-Parent,HOME_FURNITURE_AND_DECOR
3,B081K6TCML,Amazon,IN,81o9EyZ-fAL,amazon.in,Solimo Plastic Multipurpose Modular Drawer; sm...,drawer modular drawer 3 rack modular drawer ki...,Plastic,Amazon Brand - Solimo,Multicolor,Amazon Brand - Solimo Plastic Multipurpose Mod...,unknown,sol_cujo_13,HOME
4,B0854774X5,Amazon,IN,81xaJCVnl3L,amazon.in,"Snug fit for Nokia 8.1, with perfect cut-outs ...",Back Cover Designer Case Designer Take It Easy...,Silicon,Amazon Brand - Solimo,Multicolor,Amazon Brand - Solimo Designer Take It Easy UV...,Nokia 8.1,UV10714-SL40617,CELLULAR_PHONE_CASE


In [21]:
df['bullet_point'][0]

'3D Printed Hard Back Case Mobile Cover for Lenovo K4 Note Easy to put & take off with perfect cutouts for volume buttons, audio & charging ports. Stylish design and appearance, express your unique personality. Extreme precision design allows easy access to all buttons and ports while featuring raised bezel to life screen and camera off flat surface. Slim Hard Back Cover No Warranty None'

In [22]:
df['item_keywords'][0]

'mobile cover back cover mobile case phone case mobile panel phone panel Lenovo mobile case Lenovo phone cover Lenovo back case hard case 3D printed mobile cover mobile cover back cover mobile case phone case mobile panel phone panel Lenovo mobile case Lenovo phone cover Lenovo back case hard case 3D printed mobile cover mobile cover back cover mobile case phone case mobile panel phone panel Lenovo mobile case Lenovo phone cover Lenovo back case hard case 3D printed mobile cover mobile cover back cover mobile case phone case mobile panel phone panel Lenovo mobile case Lenovo phone cover Lenovo back case hard case 3D printed mobile cover mobile cover back cover mobile case phone case mobile panel phone panel Lenovo mobile case Lenovo phone cover Lenovo back case hard case 3D printed mobile cover mobile cover back cover mobile case phone case mobile panel phone panel Lenovo mobile case Lenovo phone cover Lenovo back case hard case 3D printed mobile cover mobile cover back cover mobile ca

In [33]:
df['item_name'][3]

'Amazon Brand - Solimo Plastic Multipurpose Modular Drawer, 3 Racks, Multicolor'

In [35]:
df['product_type'][0]

'CELLULAR_PHONE_CASE'

## postgres

In [45]:
import psycopg2
from psycopg2.extras import execute_values


conn = psycopg2.connect(
    host="127.0.0.1",
    port=5432,
    dbname="Ecommerce_DB",
    user="postgres",      # or admin
    password="postgres"
)

cursor = conn.cursor()



In [46]:
df.describe()

,item_id,marketplace,country,main_image_id,brand,color,material,product_type,item_name,bullet_point,item_keywords
count,147702,147702,147702,147702,147702,147702,147702,147702,147702,147702,147702
unique,145615,7,20,123512,408,4762,1321,576,115915,33208,44865
top,B00Q6EY5UU,Amazon,IN,61OnkGVhMWL,Amazon Brand - Solimo,unknown,unknown,CELLULAR_PHONE_CASE,unknown,unknown,unknown
freq,5,141472,76442,742,67543,49199,100763,64853,24968,34221,40136


In [47]:
CREATE_TABLE_SQL = """
CREATE TABLE IF NOT EXISTS products (
    item_id TEXT PRIMARY KEY,

    -- Metadata
    marketplace TEXT,
    country TEXT,

    -- UI
    main_image_id TEXT,

    -- Filters
    brand TEXT,
    color TEXT,
    material TEXT,
    product_type TEXT,

    -- Search text (RAW)
    item_name TEXT,
    bullet_point TEXT,
    item_keywords TEXT
);
"""

cursor.execute(CREATE_TABLE_SQL)

conn.commit()
print("✓ Table `products` is ready")


✓ Table `products` is ready


In [48]:
df = df[
        [
            "item_id",
            "marketplace",
            "country",
            "main_image_id",
            "brand",
            "color",
            "material",
            "product_type",
            "item_name",
            "bullet_point",
            "item_keywords",
        ]
    ]

In [49]:
insert_query = """
        INSERT INTO products (
            item_id,
            marketplace,
            country,
            main_image_id,
            brand,
            color,
            material,
            product_type,
            item_name,
            bullet_point,
            item_keywords
        )
        VALUES %s
        ON CONFLICT (item_id) DO NOTHING;
    """

In [50]:
records = df.to_records(index=False).tolist()
execute_values(cursor, insert_query, records)

In [51]:
conn.commit()
print(f"✓ Inserted {len(records)} rows into products")


✓ Inserted 147702 rows into products


In [57]:
cursor.execute("SELECT * FROM products")
rows = cursor.fetchall()


In [58]:
len(rows)

145615

In [59]:
rows[0]

('B07T6RZ2CM',
 'Amazon',
 'IN',
 '71dZhpsferL',
 'Amazon Brand - Solimo',
 'Others',
 'unknown',
 'CELLULAR_PHONE_CASE',
 'Amazon Brand - Solimo Designer Couples Sitting at Dark 3D Printed Hard Back Case Mobile Cover for Lenovo K4 Note',
 '3D Printed Hard Back Case Mobile Cover for Lenovo K4 Note Easy to put & take off with perfect cutouts for volume buttons, audio & charging ports. Stylish design and appearance, express your unique personality. Extreme precision design allows easy access to all buttons and ports while featuring raised bezel to life screen and camera off flat surface. Slim Hard Back Cover No Warranty None',
 'mobile cover back cover mobile case phone case mobile panel phone panel Lenovo mobile case Lenovo phone cover Lenovo back case hard case 3D printed mobile cover mobile cover back cover mobile case phone case mobile panel phone panel Lenovo mobile case Lenovo phone cover Lenovo back case hard case 3D printed mobile cover mobile cover back cover mobile case phone c

In [60]:
cursor.close()
conn.close()